# [E-09] Movielens 영화 추천 실습

## 1. 데이터 준비와 전처리

In [1]:
import os
import pandas as pd
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [4]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


## 2. 데이터 분석

### 2.1 ratings에 있는 유니크한 영화 개수

In [6]:
ratings['movie_id'].nunique()

3628

### 2.2 ratings에 있는 유니크한 사용자 수

In [7]:
ratings['user_id'].nunique()

6039

### 2.2 가장 인기 있는 영화 30개(인기순)

In [20]:
movie_count = ratings.groupby('movie_id')['user_id'].count()
best_movie = movie_count.sort_values(ascending=False).head(30)

In [21]:
best_movie = best_movie.reset_index()
pd.merge(best_movie, movies)

,movie_id,user_id,title,genre
0,2858,3211,American Beauty (1999),Comedy|Drama
1,260,2910,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
2,1196,2885,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
3,1210,2716,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
4,2028,2561,Saving Private Ryan (1998),Action|Drama|War
5,589,2509,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
6,593,2498,"Silence of the Lambs, The (1991)",Drama|Thriller
7,1198,2473,Raiders of the Lost Ark (1981),Action|Adventure
8,1270,2460,Back to the Future (1985),Comedy|Sci-Fi
9,2571,2434,"Matrix, The (1999)",Action|Sci-Fi|Thriller


## 3 데이터 전처리

### 3.1 데이터 합치기

In [53]:
data = pd.merge(ratings, movies)
data = data.drop(['movie_id','timestamp'], axis=1)
data

,user_id,count,title,genre
0,1,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,5,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,4,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,4,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,5,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...
836473,5851,5,One Little Indian (1973),Comedy|Drama|Western
836474,5854,4,Slaughterhouse (1987),Horror
836475,5854,3,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,4,"Five Wives, Three Secretaries and Me (1998)",Documentary


### 3.2 내가 선호하는 영화

In [54]:
# 넣다보니 다 로맨스군요,, 이거 다 재미있습니다 
my_favorite = ['Toy Story (1995)', 'Pretty Woman (1990)', 'Notting Hill (1999)', 'Breakfast at Tiffany\'s (1961)', 'Ghost (1990)']

# 'mink'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['mink']*5, 'title': my_favorite, 'count':[5]*5})

if not data.isin({'user_id':['mink']})['user_id'].any():    # user_id에 'mink'이라는 데이터가 없다면
    data = data.append(my_playlist)                          # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.


,user_id,count,title,genre
836473,5851,5,One Little Indian (1973),Comedy|Drama|Western
836474,5854,4,Slaughterhouse (1987),Horror
836475,5854,3,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,4,"Five Wives, Three Secretaries and Me (1998)",Documentary
836477,5948,5,Identification of a Woman (Identificazione di ...,Drama
0,mink,5,Toy Story (1995),NaN
1,mink,5,Pretty Woman (1990),NaN
2,mink,5,Notting Hill (1999),NaN
3,mink,5,Breakfast at Tiffany's (1961),NaN
4,mink,5,Ghost (1990),NaN


In [55]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = data['user_id'].unique()
artist_unique = data['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(artist_unique)}

In [56]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['mink'])    # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 
print(movie_to_idx['Pretty Woman (1990)'])

6039
336


In [57]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = data['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_movie_data
else:
    print('title column indexing Fail!!')

data

user_id column indexing OK!!
title column indexing OK!!


,user_id,count,title,genre
0,0,5,0,Drama
1,1,5,0,Drama
2,2,4,0,Drama
3,3,4,0,Drama
4,4,5,0,Drama
...,...,...,...,...
0,6039,5,40,NaN
1,6039,5,336,NaN
2,6039,5,1151,NaN
3,6039,5,566,NaN


## 4. CSR matrix

In [58]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_title = data['title'].nunique()

csr_data = csr_matrix((data['count'], (data.user_id, data.title)), shape=(num_user, num_title))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5. 학습 시키기 (Training)

In [59]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [60]:
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [61]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [62]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [63]:
mink, PrettyWoman = user_to_idx['mink'], movie_to_idx['Pretty Woman (1990)']
mink_vector, PrettyWoman_vector = als_model.user_factors[zimin], als_model.item_factors[black_eyed_peas]

In [64]:
mink_vector

array([ 0.5945224 , -0.03569415, -0.42311013, -0.11204351,  0.44031662,
        0.4154035 ,  0.08935115,  1.2173897 , -0.86570483,  0.2347211 ,
        0.2893125 , -0.70159197, -0.20880243, -0.21866286,  0.50983834,
        0.28204244, -0.05481227,  0.09702431,  0.8913216 ,  0.03009138,
       -0.03872857, -0.42032975, -0.01920801, -0.82574797, -0.7748136 ,
        0.05667796, -0.2868886 , -0.10137396, -0.45040038,  0.206628  ,
       -0.72492   ,  0.36847025, -1.0645248 ,  0.22535482,  0.5096727 ,
       -0.4211971 ,  0.13075224,  0.11245179, -0.28694907,  1.0436443 ,
       -0.500026  , -0.32849747, -0.12172833,  0.14642467, -0.23427612,
       -0.98694986,  0.5600296 ,  0.25526416, -0.5742413 ,  0.3560149 ,
       -0.07908549, -0.2887815 , -0.01380148,  0.35808843,  0.46262   ,
        0.28610447,  0.17960714, -0.1980718 ,  0.56683534,  0.34751737,
       -0.04391448, -0.8810737 ,  0.626443  , -0.0779124 ,  0.37787357,
        0.12122978, -0.07306499,  0.13135213,  0.6429734 ,  0.79

In [65]:
PrettyWoman_vector

array([ 0.0111395 ,  0.01676977,  0.00716284, -0.00844191,  0.0065935 ,
        0.0253495 , -0.01861889,  0.03521147, -0.02149946, -0.00121376,
       -0.00034977, -0.02197885,  0.00199578,  0.01402495,  0.00428218,
        0.00781113,  0.02215931, -0.01250852,  0.00370657,  0.00603813,
        0.00860093, -0.01357391,  0.0180407 , -0.00977964, -0.0274906 ,
        0.0003307 , -0.00336456,  0.00335022,  0.00191691,  0.00318918,
       -0.0027618 ,  0.01433651, -0.00343738,  0.00821603,  0.01928795,
        0.00345864,  0.03451899, -0.00118674,  0.01171053,  0.02341322,
       -0.00776284, -0.00949393, -0.00578183,  0.01700256,  0.00732155,
       -0.01223066,  0.01968187,  0.01559715,  0.00403039,  0.0077716 ,
        0.01450721, -0.00259867, -0.00213894,  0.00809579,  0.00604162,
        0.01131814,  0.00019166,  0.00255294,  0.01633239, -0.00210829,
        0.02230552, -0.00179425,  0.00950474, -0.00270013,  0.01472843,
       -0.00352157, -0.00611226,  0.01129481,  0.03784478,  0.01

In [66]:
np.dot(mink_vector, PrettyWoman_vector)

0.47841865

In [67]:
BaB = movie_to_idx['Beauty and the Beast (1991)']
BaB_vector = als_model.item_factors[BaB]
np.dot(mink_vector, BaB_vector)

0.05679646

## 6. 영화 추천

In [71]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}

def get_similar_artist(title: str):
    movie_id = movie_to_idx[title]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [72]:
get_similar_artist('Pretty Woman (1990)')

['Pretty Woman (1990)',
 'Sleepless in Seattle (1993)',
 "My Best Friend's Wedding (1997)",
 "You've Got Mail (1998)",
 'Ghost (1990)',
 'Notting Hill (1999)',
 'While You Were Sleeping (1995)',
 'Truth About Cats & Dogs, The (1996)',
 'Mrs. Doubtfire (1993)',
 'Sabrina (1995)']

In [77]:
user = user_to_idx['mink']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
[idx_to_movie[i[0]] for i in movie_recommended]

['Sleepless in Seattle (1993)',
 'Forrest Gump (1994)',
 'Toy Story 2 (1999)',
 "You've Got Mail (1998)",
 "My Best Friend's Wedding (1997)",
 'Groundhog Day (1993)',
 'Shakespeare in Love (1998)',
 'Titanic (1997)',
 'Four Weddings and a Funeral (1994)',
 'Babe (1995)',
 'Dave (1993)',
 'American President, The (1995)',
 'While You Were Sleeping (1995)',
 "Bug's Life, A (1998)",
 'Wedding Singer, The (1998)',
 'Clueless (1995)',
 'Mrs. Doubtfire (1993)',
 'Sabrina (1995)',
 'Sixth Sense, The (1999)',
 'Truth About Cats & Dogs, The (1996)']

['Sleepless in Seattle (1993)',
 'Forrest Gump (1994)',
 'Toy Story 2 (1999)',
 "You've Got Mail (1998)",
 "My Best Friend's Wedding (1997)",
 'Groundhog Day (1993)',
 'Shakespeare in Love (1998)',
 'Titanic (1997)',
 'Four Weddings and a Funeral (1994)',
 'Babe (1995)',
 'Dave (1993)',
 'American President, The (1995)',
 'While You Were Sleeping (1995)',
 "Bug's Life, A (1998)",
 'Wedding Singer, The (1998)',
 'Clueless (1995)',
 'Mrs. Doubtfire (1993)',
 'Sabrina (1995)',
 'Sixth Sense, The (1999)',
 'Truth About Cats & Dogs, The (1996)']